<a href="https://colab.research.google.com/github/bloodfield2/ProyectoVision/blob/main/Proyecto_de_visi%C3%B3n.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Proyecto de Visión 
#     Practical Self Calibration Using a Planar Scene
## Estudiantes
Chin Parra Keny

Chávez Aparicio Edgar Iván

##    Guía

1. cargar los siguientes archivos a /content/:
+ picture_2022-03-09_14-10-45.jpg
+ picture_2022-03-09_14-10-49.jpg
+ rookie.py


#    EN PROGRESO:

Se está probando el código para el caso de un solo par de imágenes para
después extender a N.

#    Instrucciones:

Trabajar las funciones de forma modular en `rookie.py`
Lo que se esté trabajando, desarrollarlo en el block de notas
Una vez terminadas las faces, hacer un módulo si está muy grande

#    TODO

##  ~Para un solo par de imágnes~
+ ~Feature Matching~
+ ~Projective reconstruction~
+ ~Projective Boundle Adjustment~
+ ~Homography based calibration~
+ ~Metric reconstruction~


##  Para N imágnes
+ Feature Matching
+ Projective reconstruction
+ Projective Boundle Adjustment 
+ Homography based calibration
+ Metric reconstruction

In [1]:
import numpy as np
import cv2 
import glob
from rookie  import *
from scipy.optimize import minimize
from scipy.optimize import minimize_scalar


In [2]:
img1 = cv2.imread('picture_2022-03-09_14-10-45.jpg',0)
img2 = cv2.imread('picture_2022-03-09_14-10-49.jpg',0)
print(img1.shape)

(480, 640)


In [3]:
# Part 1: Feature Matching
points1, points2 = Features(img1,img2)
print(points1.shape,points2.shape)


(75, 2) (75, 2)


In [4]:
# Part 2: First Proyective resconstruction
h, status = cv2.findHomography(points1, points2, cv2.RANSAC,0.5)
print(h)

[[ 2.37395862e+00  2.68222567e-01 -2.16694685e+02]
 [ 2.30522793e-01  1.63172925e+00 -3.13791974e+01]
 [ 2.38005858e-03  1.07947996e-04  1.00000000e+00]]


In [5]:
# Part 3: Projective Boundle adjustment

x = np.array([1,1,1,1]) # starting point
center = np.array(list(img1.shape))/2
center = center.reshape((2,1))
args = (points1,points2,h,center)
#PBA(x,points1,points2,h,np.array(list(img1.shape))/2.0 )
xmin = minimize(PBA, x,args = args, method='nelder-mead',options={'maxiter' :2000,'disp': True},tol = 10**-12)
print(xmin)
xmin = xmin['x']
p0 = xmin[:2]
d = xmin[2:]

Optimization terminated successfully.
         Current function value: 17281.220054
         Iterations: 890
         Function evaluations: 1684
 final_simplex: (array([[ 9.60294230e-01,  1.28615744e+00,  2.31810265e-08,
        -1.49678420e-13],
       [ 9.60294230e-01,  1.28615744e+00,  2.31810265e-08,
        -1.49678420e-13],
       [ 9.60294230e-01,  1.28615744e+00,  2.31810265e-08,
        -1.49678420e-13],
       [ 9.60294230e-01,  1.28615744e+00,  2.31810265e-08,
        -1.49678420e-13],
       [ 9.60294230e-01,  1.28615744e+00,  2.31810265e-08,
        -1.49678420e-13]]), array([17281.22005413, 17281.22005413, 17281.22005413, 17281.22005413,
       17281.22005413]))
           fun: 17281.220054126083
       message: 'Optimization terminated successfully.'
          nfev: 1684
           nit: 890
        status: 0
       success: True
             x: array([ 9.60294230e-01,  1.28615744e+00,  2.31810265e-08, -1.49678420e-13])


In [8]:
#   part 4: Homography self calibration
#h /= norm(h)
args = (h)
xmin = minimize_scalar(residuals, args = args,tol = 10**-7)
print(xmin)
xmin = xmin['x']

h /= norm(h)
args = (h)
x = np.array([xmin,p0[0],xmin,p0[1],0,0,1])
xmin = minimize(FNF, x,args = args, method='nelder-mead',options={'disp': True},tol = 10**-7)
print(xmin)
xmin = xmin['x']
f = np.array([xmin[0],xmin[2]])
p0 = np.array([xmin[1],xmin[3]]).reshape((2,1))
n0 = xmin[4:]

     fun: 6.513258801718936e-05
    nfev: 17
     nit: 13
 success: True
       x: 1.0817795561482467e-11
 final_simplex: (array([[ 1.09943756e-11,  3.92995811e-01, -1.51384677e-19,
         1.38313503e+00,  5.17770068e-03,  1.61501522e-20,
         1.98881000e+00],
       [ 1.09943756e-11,  3.92995814e-01, -9.61322996e-20,
         1.38313503e+00,  5.17770065e-03,  1.02556734e-20,
         1.98881000e+00],
       [ 1.09943756e-11,  3.92995811e-01, -1.45841977e-19,
         1.38313503e+00,  5.17770068e-03,  1.55588403e-20,
         1.98881000e+00],
       [ 1.09943756e-11,  3.92995812e-01, -1.32415897e-19,
         1.38313503e+00,  5.17770067e-03,  1.41265128e-20,
         1.98881000e+00],
       [ 1.09943756e-11,  3.92995811e-01, -1.40928356e-19,
         1.38313503e+00,  5.17770068e-03,  1.50346406e-20,
         1.98881000e+00],
       [ 1.09943756e-11,  3.92995810e-01, -1.65688683e-19,
         1.38313503e+00,  5.17770069e-03,  1.76761499e-20,
         1.98881000e+00],
       [ 1.09

In [9]:
#   part 5: Metric Reconstruction
K = np.array([[xmin[0],0,xmin[1]],[0,xmin[2],xmin[3]],[0,0,1]])
x = np.concatenate([K.reshape((9)),d])    #   K d 
args = (n0,h,p0,points1,points2)
res = MRF(x,*args)

print(res)

19965.021469655385


In [6]:
#   Test
m = points1.shape[0]
test_pref = np.concatenate([points1.T,np.ones((1,m))])
test_pref = h@test_pref 
test_pref = test_pref[:2,:]/test_pref[2,:]
test_pref = Distorted(test_pref,xmin[:2].reshape((2,1)),xmin[2:4])
norms = np.linalg.norm(test_pref-points2,axis = 1)
print(norms.sum(),norms.mean(),norms.max())
test_pref = np.concatenate([points1.T,np.ones((1,m))])
test_pref = h@test_pref 
test_pref = test_pref[:2,:]/test_pref[2,:]
norms = np.linalg.norm(test_pref.T-points2,axis = 1)
print(norms.sum(),norms.mean(),norms.max())

74.27147400871847 0.9902863201162463 5.761299192945103
74.27576287630956 0.9903435050174607 5.767173511835501
